In [4]:
from slop_pre_processing import TfidfVectorizer, VectorizerParams, __version__

print(__version__)

vectorizer_params = VectorizerParams(ngram_range=(1, 2), min_df=1)


vectorizer = TfidfVectorizer.fit(
    texts=["this is a sample", "this is another example sample"],
    params=vectorizer_params,
)

0.1.0


In [ ]:
sparse = vectorizer.transform(["this is a sample and ma"])

In [6]:
vectorizer.vocabulary()

{' a sample': 2,
 'this': 10,
 ' a': 1,
 ' another': 3,
 ' sample': 0,
 'this is': 11,
 ' is another': 7,
 ' another example': 4,
 ' example': 8,
 ' is': 5,
 ' is a': 6,
 ' example sample': 9}